In [14]:
import pandas as pd  
import numpy as np  
import statsmodels.tools.tools as stattools  
from sklearn.tree import DecisionTreeClassifier  

In [15]:
#from google.colab import files
#uploaded = files.upload()

clothing_train = pd.read_csv("clothing_data_driven_training (1)", sep="\t") 
clothing_test = pd.read_csv("clothing_data_driven_test", sep="\t") 

In [16]:
clothing_test

,Days since Purchase,Number of Purchase Visits,Response,Sales per Visit
0,208,2,0,184.230000
1,64,5,1,104.240000
2,23,12,1,138.621667
3,144,7,1,90.297143
4,118,1,0,172.990000
...,...,...,...,...
11549,186,4,0,26.247500
11550,137,1,0,139.970000
11551,75,2,0,199.940000
11552,257,1,0,38.490000


In [17]:
clothing_train

,Days since Purchase,Number of Purchase Visits,Response,Sales per Visit
0,6,4,1,64.500000
1,327,2,0,38.500000
2,66,8,0,105.757500
3,49,1,0,87.440000
4,26,2,0,60.000000
...,...,...,...,...
17240,322,2,0,19.500000
17241,53,6,0,46.765000
17242,7,11,0,97.590909
17243,24,9,1,141.662222


Using the training data set, create a C5.0 model (Model 1) to predict a customer's Response using Days since Purchase, # of Purchase Visits, and Sales per Visit. Obtain the predicted responses. 

In [18]:
# save target variable  
y = clothing_train[['Response']]  
  
# create dataset of predictors  
X = clothing_train[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']]  
  
# train C5.0 model  
c50_01 = DecisionTreeClassifier(criterion="entropy", min_samples_split=75, max_leaf_nodes=5).fit(X,y)  
  
# use model to predict on test data  
c50_01_predict = c50_01.predict(clothing_test[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']])  


Evaluate Model 1 using the test data set. Construct a contingency table to compare the actual and predicted values of Response. 

In [19]:
# construct contingency table  
con_table = pd.crosstab(clothing_test['Response'], c50_01_predict)  
con_table

col_0,0,1
Response,,
0,9173,441
1,1396,544


In [20]:
def model_eval(matrix,model_name):  
#from a dataframe and iloc is Purely integer-location based indexing for selection by position.
    tn = matrix.iloc[0,0]  
    tp = matrix.iloc[1,1]  
    fn = matrix.iloc[1,0]  
    fp = matrix.iloc[0,1]  
#some formulas
    tap = fn+tp  
    tan = tn+fp  
    tpn = tn+fn  
    tpp = fp+tp  
    precision = tp/tpp  
    recall = tp/tap  
    total = tn+tp+fn+fp  
    data = [  
	            round((tp+tn)/total,4),  #accuracy
	            round(1-((tp+tn)/total),4),  #error rate
	            round(tp/tap,4),  #Snesitivity
	            round(tn/tan,4),  #Specificity / Recall
	            round(precision,4),  #Precision
	            round(2 * (precision * recall) / (precision + recall),4),	#F1
	            round(5 * (precision * recall) / ((4 * precision) + recall),4),	#F2
	            round(1.25 * (precision * recall) / ((.25 * precision) + recall),4),	#F3  
	            'Na',  
	            'Na'  
	            ]  
    return(  
	            pd.DataFrame(data, columns=[model_name],   
	                         index=['Accuracy','Error Rate','Sensitivity','Specificity','Precision','F1','F2','F3','overall_cost','profit_per_customer'])  
	            )  
	      
Model_1 = model_eval(con_table, model_name = 'Model_1')  
Model_1  


,Model_1
Accuracy,0.841
Error Rate,0.159
Sensitivity,0.2804
Specificity,0.9541
Precision,0.5523
F1,0.372
F2,0.311
F3,0.4626
overall_cost,Na
profit_per_customer,Na


Create a cost matrix, called the 4x cost matrix, that specifies a false positive is four times as bad as a false negative. 

In [21]:
cost_matrix_4x = {0:1, 1:4}  

Using the training data set, build a C5.0 model (Model 2) to predict a customer's Response using Days since Purchase, # of Purchase Visits, and Sales per Visit, using the 4x cost matrix. 

In [22]:
# train C5.0 model  
c50_02 = DecisionTreeClassifier(criterion="entropy", min_samples_split=75, max_leaf_nodes=5, class_weight=cost_matrix_4x).fit(X,y)  

Evaluate your predictions from Model 2 using the actual response values from the test data set. Add Overall Model Cost and Profit per Customer to the Model Evaluation Table. Calculate all the measures from the Model Evaluation Table. 

In [23]:
# use model to predict on test data  
c50_02_predict = c50_02.predict(clothing_test[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']])  
 
# construct contingency table  
con_table_2 = pd.crosstab(clothing_test['Response'], c50_02_predict)
con_table_2

col_0,0,1
Response,,
0,7004,2610
1,591,1349


In [24]:

  
def model_eval_cost(matrix,model_name,fn_cost,tn_cost,fp_cost,tp_cost):  
    tn = matrix.iloc[0,0]  
    tp = matrix.iloc[1,1]  
    fn = matrix.iloc[1,0]  
    fp = matrix.iloc[0,1]  
    tap = fn+tp  
    tan = tn+fp  
    tpn = tn+fn  
    tpp = fp+tp  
    precision = tp/tpp  if tpp != 0 else 0  
    recall = tp/tap  if tap != 0 else 0  
    total = tn+tp+fn+fp  
    data = [  
            round((tp+tn)/total,4),  
            round(1-((tp+tn)/total),4),  
            round(tp/tap,4),  
            round(tn/tan,4),  
            round(precision,4),  
            round(2 * (precision * recall) / (precision + recall),4) if (precision + recall) !=0 else 'Na',  
            round(5 * (precision * recall) / ((4 * precision) + recall),4) if (precision + recall) !=0 else 'Na',  
            round(1.25 * (precision * recall) / ((.25 * precision) + recall),4) if (precision + recall) !=0 else 'Na',  
            round((fn*fn_cost)+(tn*tn_cost)+(fp*fp_cost)+(tp*tp_cost),2),  
            round(((fn*fn_cost)+(tn*tn_cost)+(fp*fp_cost)+(tp*tp_cost))/total,2)  
            ]  
  
    return(  
            pd.DataFrame(data, columns=[model_name],  
                         index=['Accuracy','Error Rate','Sensitivity','Specificity','Precision','F1','F2','F3','overall_cost','profit_per_customer'])  
            )  
  
Model_2 = model_eval_cost(con_table_2, model_name = 'Model_2', fn_cost=0, tn_cost=0, fp_cost=10, tp_cost=-40)  
# Model_2 = model_eval_cost(con_table_2, model_name = 'Model_2', fn_cost=4, tn_cost=0, fp_cost=1, tp_cost=0)  
Model_2  


,Model_2
Accuracy,0.7230
Error Rate,0.2770
Sensitivity,0.6954
Specificity,0.7285
Precision,0.3407
F1,0.4574
F2,0.5756
F3,0.3794
overall_cost,-27860.0000
profit_per_customer,-2.4100
